In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import csv as csv
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from citipy import citipy

In [2]:
df_1=pd.read_csv("Resources/Milton_Avg_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_2=pd.read_csv("Resources/Milton_Avg_Sale_Price_updated.csv", encoding='ISO-8859-1')
df_3=pd.read_csv("Resources/Milton_Median_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_4=pd.read_csv("Resources/Milton_Median_Sale_Price_updated.csv", encoding='ISO-8859-1')

In [3]:
milton_df_2021 = pd.read_csv("Resources/CensusProfile2021-ProfilRecensement2021-20231206084412.csv", encoding='ISO-8859-1')
milton_df_2021.head()

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
2,Population and dwellings,"Population percentage change, 2016 to 2021",20.7
3,Population and dwellings,Total private dwellings,41000.0
4,Population and dwellings,Private dwellings occupied by usual residents,40038.0


In [4]:
milton_df_2021.columns

Index(['Topic', 'Characteristic', 'Total'], dtype='object')

In [5]:
milton_df_2021_cleaning = milton_df_2021.drop(milton_df_2021[milton_df_2021['Topic'] == 'Population and dwellings'].index[[2, 3, 4, 5, 6]])
milton_df_2021_cleaning

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
8,Age characteristics,0 to 14 years,31980.0
9,Age characteristics,0 to 4 years,8875.0
...,...,...,...
223,Labour force status,Unemployed,8035.0
224,Labour force status,Not in the labour force,28790.0
225,Labour force status,Participation rate,71.1
226,Labour force status,Employment rate,63.0


In [6]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Age characteristics'].index

Index([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      dtype='int64')

In [7]:
indices_to_drop = [8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')

milton_df_2021_cleaning.head()

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0


In [8]:
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Household and dwelling characteristics']
milton_df_2021_cleaning.head()

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0


In [9]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Income of households in 2020'].index


Index([ 57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115],
      dtype='int64')

In [10]:
indices_to_drop = [57, 60,  61,  62,  63,  64,  65,  66,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning.head()

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0


In [11]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Household characteristics'].index

Index([116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201],
      dtype='int64')

In [48]:

milton_df_2021_cleaning

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0
58,Income of households in 2020,Median total income of household in 2020 ($),126000.0
59,Income of households in 2020,Median after-tax income of household in 2020...,107000.0
67,Income of households in 2020,Average total income of household in 2020 ($),142600.0
68,Income of households in 2020,Average after-tax income of household in 202...,116300.0
116,Household characteristics,Total - Private households by tenure - 25% sam...,40035.0


In [49]:
indices_to_drop = [119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171,
       172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185,
       186, 187, 192, 193, 194, 195, 196, 197, 198, 199,
       200, 201]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0
58,Income of households in 2020,Median total income of household in 2020 ($),126000.0
59,Income of households in 2020,Median after-tax income of household in 2020...,107000.0
67,Income of households in 2020,Average total income of household in 2020 ($),142600.0
68,Income of households in 2020,Average after-tax income of household in 202...,116300.0
116,Household characteristics,Total - Private households by tenure - 25% sam...,40035.0


In [14]:
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Mobility status 1 year ago']
milton_df_2021_cleaning = milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] != 'Mobility status 5 years ago']
milton_df_2021_cleaning

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0
58,Income of households in 2020,Median total income of household in 2020 ($),126000.0
59,Income of households in 2020,Median after-tax income of household in 2020...,107000.0
67,Income of households in 2020,Average total income of household in 2020 ($),142600.0
68,Income of households in 2020,Average after-tax income of household in 202...,116300.0
116,Household characteristics,Total - Private households by tenure - 25% sam...,40035.0


In [15]:
milton_df_2021_cleaning.loc[milton_df_2021_cleaning['Topic'] == 'Labour force status'].index

Index([220, 221, 222, 223, 224, 225, 226, 227], dtype='int64')

In [16]:
indices_to_drop = [220, 221, 222, 223, 224, 225, 226]
milton_df_2021_cleaning = milton_df_2021_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2021_cleaning

,Topic,Characteristic,Total
0,Population and dwellings,"Population, 2021",132979.0
1,Population and dwellings,"Population, 2016",110128.0
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0
12,Age characteristics,15 to 64 years,87770.0
23,Age characteristics,65 years and over,13225.0
58,Income of households in 2020,Median total income of household in 2020 ($),126000.0
59,Income of households in 2020,Median after-tax income of household in 2020...,107000.0
67,Income of households in 2020,Average total income of household in 2020 ($),142600.0
68,Income of households in 2020,Average after-tax income of household in 202...,116300.0
116,Household characteristics,Total - Private households by tenure - 25% sam...,40035.0


In [17]:
#milton_housing_df = pd.read_csv("Resources/TableExport (1).csv", encoding='ISO-8859-1',sep=';')
#milton_housing_df.columns

In [19]:
milton_df_2016 = pd.read_csv("Resources/CensusProfile2016-ProfilRecensement2016-20231207084508.csv", encoding='ISO-8859-1')
milton_df_2016['Topic'].unique()

array(['Population and dwellings', 'Age characteristics',
       'Household and dwelling characteristics', 'Marital status',
       'Family characteristics', 'Household type',
       'Knowledge of official languages',
       'First official language spoken', 'Mother tongue',
       'Language spoken most often at home',
       'Other language spoken regularly at home',
       'Income of individuals in 2015', 'Income of households in 2015',
       'Income of economic families in 2015', 'Low income in 2015',
       'Knowledge of languages', 'Citizenship',
       'Immigrant status and period of immigration', 'Age at immigration',
       'Immigrants by selected place of birth',
       'Recent immigrants by selected places of birth',
       'Generation status', 'Admission category and applicant type',
       'Aboriginal population', 'Visible minority population',
       'Ethnic origin population', 'Household characteristics',
       ' Highest certificate; diploma or degree',
       'Major fi

In [27]:
milton_df_2016_cleaning = milton_df_2016.loc[milton_df_2016['Topic'] != 'Household and dwelling characteristics']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'First official language spoken']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Marital status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Other language spoken regularly at home']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Family characteristics']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Knowledge of official languages']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mother tongue']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Language spoken most often at home']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Income of individuals in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Income of economic families in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Low income in 2015']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Knowledge of languages']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Citizenship']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Immigrant status and period of immigration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Age at immigration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Immigrants by selected place of birth']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Recent immigrants by selected places of birth']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Generation status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Admission category and applicant type']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Aboriginal population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Visible minority population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Ethnic origin population']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Highest certificate; diploma or degree']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Major field of study - Classification of Instructional Programs (CIP) 2016']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Location of study compared with province or territory of residence with countries outside Canada']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Occupation - National Occupational Classification (NOC) 2016']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Industry - North American Industry Classification System (NAICS) 2012']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Place of work status']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Commuting destination']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Main mode of commuting']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Commuting duration']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Time leaving for work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Language used most often at work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Other language used regularly at work']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mobility status - Place of residence 1 year ago']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Mobility status - Place of residence 5 years ago']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Household type']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Work activity during the reference year']
milton_df_2016_cleaning = milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] != 'Class of worker']
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
1,Population and dwellings,Population; 2011,NaN,84362.0
2,Population and dwellings,Population percentage change; 2011 to 2016,NaN,30.5
3,Population and dwellings,Total private dwellings,NaN,35537.0
4,Population and dwellings,Private dwellings occupied by usual residents,NaN,34257.0


In [28]:
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(milton_df_2016_cleaning[milton_df_2016_cleaning['Topic'] == 'Population and dwellings'].index[[1, 2, 3, 4, 5, 6]])
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
8,Age characteristics,0 to 14 years,NaN,28405.0
9,Age characteristics,0 to 4 years,NaN,9315.0
10,Age characteristics,5 to 9 years,NaN,10505.0


In [29]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Age characteristics'].index


Index([ 7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
      dtype='int64')

In [30]:
indices_to_drop = [8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
740,Income of households in 2015,Total - Income statistics in 2015 for private ...,NaN,34255.0


In [31]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Income of households in 2015'].index


Index([740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767,
       768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781,
       782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795,
       796, 797, 798],
      dtype='int64')

In [32]:

indices_to_drop = [740, 743, 744, 745, 746, 747, 748, 749, 752, 753,
       754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767,
       768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781,
       782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795,
       796, 797, 798]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.tail()

,Topic,Characteristics,Unnamed: 2,Total
1867,Labour force status,Unemployed,NaN,3330.0
1868,Labour force status,Not in the labour force,NaN,20740.0
1869,Labour force status,Participation rate,NaN,74.2
1870,Labour force status,Employment rate,NaN,70.1
1871,Labour force status,Unemployment rate,NaN,5.6


In [33]:
milton_df_2016_cleaning.loc[milton_df_2016_cleaning['Topic'] == 'Household characteristics'].index


Index([1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627,
       1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639,
       1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663,
       1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1675,
       1676, 1677, 1678, 1679, 1680, 1681],
      dtype='int64')

In [35]:
indices_to_drop = [1619, 1620, 1621, 1622, 1623, 1624, 1625, 1626, 1627,
       1628, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638, 1639,
       1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651,
       1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663,
       1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674, 1677, 1678, 1679]
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.head()

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0


In [36]:
indices_to_drop = range (1682,1712)
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning.head(60)

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0
742,Income of households in 2015,Median after-tax income of households in 201...,NaN,88711.0
750,Income of households in 2015,Average total income of households in 2015 ($),NaN,120157.0
751,Income of households in 2015,Average after-tax income of households in 20...,NaN,97783.0
1616,Household characteristics,Total - Private households by tenure - 25% sam...,NaN,34260.0
1617,Household characteristics,Owner,NaN,29415.0


In [37]:
indices_to_drop = range (1864,1871)
milton_df_2016_cleaning = milton_df_2016_cleaning.drop(indices_to_drop, errors='ignore')
milton_df_2016_cleaning

,Topic,Characteristics,Unnamed: 2,Total
0,Population and dwellings,Population; 2016,NaN,110128.0
7,Age characteristics,Total - Age groups and average age of the popu...,NaN,110125.0
12,Age characteristics,15 to 64 years,NaN,71580.0
23,Age characteristics,65 years and over,NaN,10145.0
741,Income of households in 2015,Median total income of households in 2015 ($),NaN,104730.0
742,Income of households in 2015,Median after-tax income of households in 201...,NaN,88711.0
750,Income of households in 2015,Average total income of households in 2015 ($),NaN,120157.0
751,Income of households in 2015,Average after-tax income of households in 20...,NaN,97783.0
1616,Household characteristics,Total - Private households by tenure - 25% sam...,NaN,34260.0
1617,Household characteristics,Owner,NaN,29415.0


In [47]:
milton_df = pd.merge(milton_df_2021_cleaning, milton_df_2016_cleaning, how='outer', left_index=True, right_index=True)
del milton_df['Unnamed: 2']
milton_df

,Topic_x,Characteristic,Total_x,Topic_y,Characteristics,Total_y
0,Population and dwellings,"Population, 2021",132979.0,Population and dwellings,Population; 2016,110128.0
1,Population and dwellings,"Population, 2016",110128.0,NaN,NaN,NaN
7,Age characteristics,Total - Age groups of the population - 100% data,132980.0,Age characteristics,Total - Age groups and average age of the popu...,110125.0
12,Age characteristics,15 to 64 years,87770.0,Age characteristics,15 to 64 years,71580.0
23,Age characteristics,65 years and over,13225.0,Age characteristics,65 years and over,10145.0
58,Income of households in 2020,Median total income of household in 2020 ($),126000.0,NaN,NaN,NaN
59,Income of households in 2020,Median after-tax income of household in 2020...,107000.0,NaN,NaN,NaN
67,Income of households in 2020,Average total income of household in 2020 ($),142600.0,NaN,NaN,NaN
68,Income of households in 2020,Average after-tax income of household in 202...,116300.0,NaN,NaN,NaN
116,Household characteristics,Total - Private households by tenure - 25% sam...,40035.0,NaN,NaN,NaN


In [63]:
milton_df_one = pd.merge(df_1, df_2, how='inner', on='Primary Year')
milton_df_one = milton_df_one.rename(columns= {'Sale Price, Average_x' : 'Average Rental Price', 'Sale Price, Average_y' :'Average Housing Selling Price'})

In [64]:
milton_df_two = pd.merge(df_3, df_4, how='inner', on='Primary Year')
milton_df_two = milton_df_two.rename(columns= {'Sale Price, Median_x' : 'Median Rental Price', 'Sale Price, Median_y' :'Median Housing Selling Price'})
milton_df_two

,Primary Year,Median Rental Price,Median Housing Selling Price
0,2016,"$2,150","$621,000"
1,2017,"$2,100","$705,000"
2,2018,"$2,200","$675,000"
3,2019,"$2,300","$725,000"
4,2020,"$2,400","$810,786"
5,2021,"$2,650","$990,000"
6,2022,"$2,800","$1,100,000"
7,2023,"$2,900","$1,015,000"


In [65]:
milton_df_2 = pd.merge(milton_df_two, milton_df_one, how='inner', on='Primary Year')
milton_df_2


,Primary Year,Median Rental Price,Median Housing Selling Price,Average Rental Price,Average Housing Selling Price
0,2016,"$2,150","$621,000","$2,821","$657,525"
1,2017,"$2,100","$705,000","$2,341","$762,677"
2,2018,"$2,200","$675,000","$2,243","$717,413"
3,2019,"$2,300","$725,000","$2,369","$767,261"
4,2020,"$2,400","$810,786","$2,624","$880,169"
5,2021,"$2,650","$990,000","$2,785","$1,104,704"
6,2022,"$2,800","$1,100,000","$2,820","$1,196,104"
7,2023,"$2,900","$1,015,000","$2,989","$1,101,957"


In [ ]:
df_1=pd.read_csv("Resources/Milton_Avg_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_2=pd.read_csv("Resources/Milton_Avg_Sale_Price_updated.csv", encoding='ISO-8859-1')
df_3=pd.read_csv("Resources/Milton_Median_Rental_Price_updated.csv", encoding='ISO-8859-1')
df_4=pd.read_csv("Resources/Milton_Median_Sale_Price_updated.csv", encoding='ISO-8859-1')

In [ ]:
1.City (row)
2.Population
3.Household Income
4.Average Rent
5.Median Rent
6.Estimated House Value
7.Unemployment Rate
8.Property Tax
9.Schools (Public, Private, Catholic, school rating) #own df
10.Leisure #own df
11.Rented Houses
12.Owned Houses
13.New Housing Built
14.Average Sale Price 
15.Year

In [ ]:
https://www12.statcan.gc.ca/census-recensement/index-eng.cfm

In [ ]:
https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart/Table?TableId=2.1.31.2&GeographyId=35&GeographyTypeId=2&DisplayAs=Table&GeograghyName=Ontario#Apartment